In [1]:
import pandas as pd, numpy as np

trip_sheet = '/DATA/TripAnalysis/nextbike_trips.csv'

#before cleaning
trip_df = pd.read_csv(trip_sheet.lstrip('/'))
print(trip_df.head(5).to_string(index=False))
print(f"Length of the dataframe before cleaning: {len(trip_df)}")

trip_df = (
    pd.read_csv(trip_sheet.lstrip('/'))
      .pipe(lambda d: d.rename(columns={c: c.strip() for c in d.columns}))
      .assign(**{
          'Duration-Minutes': lambda d: pd.to_numeric(d['Duration-Minutes'], errors='coerce'),
          'Rental-Lat':       lambda d: pd.to_numeric(d['Rental-Lat'], errors='coerce'),
          'Rental-Lng':       lambda d: pd.to_numeric(d['Rental-Lng'], errors='coerce'),
          'Return-Lat':       lambda d: pd.to_numeric(d['Return-Lat'], errors='coerce'),
          'Return-Lng':       lambda d: pd.to_numeric(d['Return-Lng'], errors='coerce'),
      })
      .loc[lambda d: ~d.apply(lambda r: r.astype(str).str.contains('unbekannt', case=False).any(), axis=1)]
      .loc[lambda d: d['Duration-Minutes'] > 2]
      .loc[lambda d: ~((d['Rental-Lat'] == d['Return-Lat']) & (d['Rental-Lng'] == d['Return-Lng']))]
      .assign(**{
          'Rental-Time': lambda d: pd.to_datetime(d['Rental-Time'], errors='coerce'),
          'weekday_idx': lambda d: d['Rental-Time'].dt.weekday,
          'is_weekend':  lambda d: d['weekday_idx'] >= 5,
          'hour':        lambda d: d['Rental-Time'].dt.hour,
      })
      .reset_index(drop=True)
)

print(trip_df.head(5).to_string(index=False))
print(f"Length of the dataframe after cleaning: {len(trip_df)}")

 Bike-Number         Rental-Time        Rental-Type                                     Rental-Location  Rental-Lat  Rental-Lng         Return-Time        Return-Type                  Return-Location  Return-Lat  Return-Lng  Duration-Minutes   Movement-Type
      100245 2025-07-23 21:56:39 Station (virtuell)                            U Schlesisches Tor (Ost)   52.500794   13.442182 2025-07-23 22:08:17 Station (virtuell)        Elsenstraße/Harzer Straße   52.485136   13.449063              11.6 Station:Station
       13167 2025-07-23 20:22:59 Station (virtuell)                           Elsenstraße/Harzer Straße   52.485136   13.449063 2025-07-23 22:08:17 Station (virtuell) Modersohnstraße (Am Rudolfplatz)   52.502492   13.456503             105.3 Station:Station
       19766 2025-07-23 21:56:31 Station (physisch) Jelbi S+U Zoologischer Garten/Jebensstraße (CHA/JE)   52.506622   13.331480 2025-07-23 22:08:26 Station (virtuell)  Franklinstraße/Glutenbergstraße   52.518675   13.326239   